In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse

In [2]:
N = 3
epsilon = 1
J = 3

pauli_z = sparse.dia_matrix(np.array([[1, 0],
                                      [0, -1]]))

In [3]:
def n_identities(n: int):
    """Gives a tensor made out of the tensor product on 'n' identity matrices.

    Args:
        n (int): the number of identity matrices

    Returns:
        np.ndarray: the resultant matrix (tensor)
    """
    return sparse.identity(2**n) if (n > 0) else 1

$$\mathbf{A}_i = \underbrace{\overbrace{\mathbf{I} \otimes \mathbf{I} \otimes \ldots}^{i - 1\ \text{operators}} \otimes \sigma^z \otimes \overbrace{\ldots \otimes \mathbf{I} \otimes \mathbf{I}}^{N - 1\ \text{operators}}}_{N\ \text{operators}}$$

In [4]:
# A list to store the operators A
A_ops = []
for i in range(1, N+1):
    A_i = sparse.kron(sparse.kron(n_identities(i-1), pauli_z), n_identities(N-i))
    A_ops.append(A_i)

A_ops = np.array(A_ops)

$$\mathbf{H} = \epsilon \sum_{i}^{N} \mathbf{A}_i +J \sum_{\left< i j \right>} \mathbf{A}_i \mathbf{A}_{i + 1}$$

In [5]:
# Creating the Hamiltonian
H = sparse.dia_matrix((2**N, 2**N), dtype=float)
for i in range(N):
    H += epsilon * A_ops[i] + J * (A_ops[i] @ A_ops[i+1 if (i+1 < N) else 0])

print("Hamiltonian =\n", H)

Hamiltonian =
   (0, 0)	12.0
  (1, 1)	-2.0
  (2, 2)	-2.0
  (3, 3)	-4.0
  (4, 4)	-2.0
  (5, 5)	-4.0
  (6, 6)	-4.0
  (7, 7)	6.0


In [6]:
# Getting the smallest eigenvalues and their corresponding eigenvectors
eig_vals, eig_vecs = sparse.linalg.eigs(H, which='SR')

# Turning (2**N, 6) eig_vecs into (6, 2**N)
eig_vecs = eig_vecs.T
# Removing unnecessary computed values
eig_vecs[abs(eig_vecs) < 1e-10] = 0
eig_vals = np.round(eig_vals, decimals=10)

# The lowest energy
E_0 = np.real(np.min(eig_vals))
# The state with the lowest energy
psi_0 = eig_vecs[eig_vals == E_0].T

psi_0_row, psi_0_col = psi_0.shape
# Degeneracy handler
if psi_0_col > 1:
    # Setting the first state as the ground state
    psi_0 = psi_0[:,0].reshape(-1, 1)
    # scipy.sparse.linalg.eigs gives only 6 eigenvalues.
    #The degeneracy can be more than that
    print("E_0 degeneracy =\n", psi_0_col, "(at least)")

print("E_0 =\n", E_0)

E_0 degeneracy =
 3 (at least)
E_0 =
 -4.0


$$\mathbf{M} = \sum_{i = 1}^{N} \mathbf{A}_i, \quad M_0 = \left< \psi_0 \right| \mathbf{M} \left| \psi_0 \right>$$

In [7]:
# Magnetization operator
M_op = A_ops.sum()
M = psi_0.conjugate().T @ M_op @ psi_0
if M.shape == (1, 1):
    M = np.real(M[0,0])

print("M =\n", M)

M =
 -0.9999999999999996
